# 🎬 SadTalker - AI Talking Avatar (Free)

Tạo video người nói từ ảnh tĩnh + audio

**⚠️ QUAN TRỌNG:**
1. Runtime → Change runtime type → **GPU (T4)**
2. Chạy **TẤT CẢ 5 CELLS** từ trên xuống dưới
3. Đợi mỗi cell chạy xong rồi mới chạy cell tiếp

---

In [ ]:
# 1️⃣ Clone SadTalker repo
!git clone https://github.com/OpenTalker/SadTalker.git
%cd SadTalker
print("✅ Clone xong!")

In [ ]:
# 2️⃣ Cài dependencies (mất 5 phút)
!pip install -r requirements.txt
print("✅ Dependencies đã cài!")

In [ ]:
# 3️⃣ Download AI models (mất 3-5 phút)
!bash scripts/download_models.sh
print("✅ Models đã tải xong!")

In [ ]:
# 4️⃣ Setup ngrok (chạy 1 lần đầu tiên)
# Lấy authtoken tại: https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok authtoken YOUR_NGROK_TOKEN_HERE
print("✅ Ngrok đã setup!")
print("💡 Thay YOUR_NGROK_TOKEN_HERE bằng token của bạn")

In [ ]:
# 5️⃣ Start API Server
!pip install flask pyngrok flask-cors -q

from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import subprocess
import os
import glob
import traceback

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ok', 'message': 'SadTalker API is running'}), 200

@app.route('/generate', methods=['POST'])
def generate():
    try:
        print("📥 Received generation request")
        
        # Validate files
        if 'image' not in request.files or 'audio' not in request.files:
            return jsonify({'error': 'Missing image or audio file'}), 400
        
        image = request.files['image']
        audio = request.files['audio']
        
        # Save files
        os.makedirs('/tmp/avatar_input', exist_ok=True)
        image_path = '/tmp/avatar_input/input.jpg'
        audio_path = '/tmp/avatar_input/input.wav'
        image.save(image_path)
        audio.save(audio_path)
        
        print(f"✅ Saved: {image_path}, {audio_path}")
        
        # Clean old results
        os.system('rm -rf /tmp/results/*')
        
        # Run SadTalker
        print("🎬 Running SadTalker...")
        cmd = [
            'python', 'inference.py',
            '--driven_audio', audio_path,
            '--source_image', image_path,
            '--result_dir', '/tmp/results',
            '--still',
            '--preprocess', 'full',
            '--enhancer', 'gfpgan'
        ]
        
        result = subprocess.run(
            cmd,
            cwd='/content/SadTalker',
            capture_output=True,
            text=True,
            timeout=300
        )
        
        if result.returncode != 0:
            error_msg = result.stderr or result.stdout or 'Unknown error'
            print(f"❌ SadTalker failed: {error_msg}")
            return jsonify({
                'error': 'SadTalker inference failed',
                'details': error_msg,
                'stdout': result.stdout,
                'stderr': result.stderr
            }), 500
        
        print("✅ SadTalker completed")
        
        # Find generated video
        videos = glob.glob('/tmp/results/**/*.mp4', recursive=True)
        if not videos:
            print("❌ No video found in output")
            return jsonify({'error': 'No video generated'}), 500
        
        video_path = videos[0]
        print(f"📤 Sending: {video_path}")
        
        return send_file(video_path, mimetype='video/mp4')
        
    except subprocess.TimeoutExpired:
        return jsonify({'error': 'Processing timeout (>5 minutes)'}), 500
    except Exception as e:
        error_detail = traceback.format_exc()
        print(f"❌ Exception: {error_detail}")
        return jsonify({
            'error': str(e),
            'traceback': error_detail
        }), 500

# Start ngrok tunnel
print("\n🚀 Starting ngrok tunnel...")
public_url = ngrok.connect(5000)

print("\n" + "="*60)
print("✅ API Server is running!")
print("="*60)
print(f"\n🌐 Public URL: {public_url}")
print("\n📋 Copy URL này và chạy lệnh trên máy local:")
print(f"   $env:COLAB_API_URL=\"{public_url}\"")
print("\n⚠️  QUAN TRỌNG: Giữ cell này chạy (đừng stop!)\n")
print("="*60)

# Start Flask server
app.run(port=5000, debug=False)